In [1]:
from __future__ import print_function
import os
import neat

import pandas as pd
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim


from explaneat.core.backprop import NeatNet
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation
from explaneat.visualization import visualize
from explaneat.core.experiment import ExperimentReporter
from explaneat.core.utility import one_hot_encode


from sklearn import datasets
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

from copy import deepcopy

import time
from datetime import datetime


import gzip
try:
    import cPickle as pickle  # pylint: disable=import-error
except ImportError:
    import pickle  # pylint: disable=import-error

In [2]:

USE_CUDA = torch.cuda.is_available()
# USE_CUDA = False
device = torch.device("cuda:1" if USE_CUDA else "cpu")


# Breast Cancer Experiment

This experiment (a) test the experimental environment, but is also to evaluate the efficacy of the ExplaNEAT algorithm. Speed is a critical factor, as well as stability of results on population size. Total run time will also be measured

First, we need to set a random seed and a total stopping point in the number of generations

In [3]:
my_random_seed = 42
random.seed(my_random_seed)

In [4]:
def one_hot_encode(vals):
    width = max(vals)
    newVals = []
    for val in vals:
        blank = [0. for _ in range(width + 1)]
        blank[val] = 1.
        newVals.append(blank)
    return np.asarray(newVals)


## Dataset

We are going to work with the Iris dataset, which will be loaded from `sklearn`. We want to characterise the efficacy of the algorithm with regards to a mostly untransformed dataset, so we will only normalise the features

In [5]:
breast_cancer = datasets.load_breast_cancer()
xs_raw = breast_cancer.data[:, :]
scaler = StandardScaler()
scaler.fit(xs_raw)
xs = scaler.transform(xs_raw)
ys = breast_cancer.target.astype(np.float32)
# ys_onehot = one_hot_encode(ys)

In [6]:
xs = torch.from_numpy(xs).to(device)
ys = torch.from_numpy(ys).to(device)

Let's have a look at the data we are working with

In [7]:
ys[18:23]

tensor([0., 1., 1., 1., 0.], device='cuda:1')

In [8]:
xs[18:20]

tensor([[ 1.6140,  0.6656,  1.5665,  1.7210,  0.1388, -0.0311,  0.7420,  1.1881,
         -0.8383, -1.2542,  1.2742, -0.3626,  1.4846,  1.5855, -0.1823, -0.3660,
          0.0669,  0.5538, -0.8454, -0.6801,  2.2884,  0.8472,  2.3691,  2.6675,
          0.8255,  0.3864,  1.2714,  1.8910, -0.2148, -0.4320],
        [-0.1668, -1.1472, -0.1857, -0.2520,  0.1017, -0.4369, -0.2782, -0.0286,
          0.2679, -0.7283, -0.4882, -0.7770, -0.4000, -0.3691,  0.4737, -0.6080,
         -0.2660,  0.2196, -0.0899, -0.5654, -0.2400, -1.0450, -0.2252, -0.2978,
          0.5099, -0.4896, -0.1592,  0.2161,  0.1233, -0.6293]],
       device='cuda:1', dtype=torch.float64)

## Performance metric

The NEAT implementation on which ExplaNEAT extends uses a single function call for evaluating fitness. Although this might be reworked for ExplaNEAT to be able to get consistency between the genome-evaluation and the backprop loss function, that can be reviewed later.

This use `Binary Cross Entropy Loss` from `PyTorch`

In [ ]:
def eval_genomes(genomes, config):
    loss = nn.BCELoss()
    loss = loss.to(device)
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        preds = []
        for xi in xs:
            preds.append(net.activate(xi))
        genome.fitness = float(1./loss(torch.tensor(preds).to(device), torch.tensor(ys)))

## Base configuration

We are going to create the base configuration according to an external configuration file. Per experiment, we will adjust this, later, but this defines the defaults across all runs.

In [ ]:
config_path = "./config-breast-cancer"
base_config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)


In [ ]:
base_config.pop_size

2

We also want to put a hard limit on how long this can go on for.

In [ ]:
maxNGenerations = 20

We will create a method to manage the instantiation of a population on the basis of a specific config.

In [ ]:
def instantiate_population(config, xs, ys, saveLocation):

    if not os.path.exists(saveLocation):
        os.makedirs(saveLocation)
        
    config.save(os.path.join(saveLocation, 'config.conf'))

    # Create the population, which is the top-level object for a NEAT run.
    p = BackpropPopulation(config, 
                            xs, 
                            ys, 
                            criterion=nn.BCELoss())

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5, filename_prefix=str(saveLocation) + "checkpoint-" ))
    bpReporter = backprop.BackpropReporter(True)
    p.add_reporter(bpReporter)
    p.add_reporter(ExperimentReporter(saveLocation))
    
    return p

# Experiment 1: Vary population size

The first experiment is going to examine the difference in run time different population sizes. 

In [ ]:
population_points = [2, 5, 10, 25, 50, 100]

In [ ]:
base_config.pop_size

2

In [ ]:
saveLocationTemplate = './../../data/experiments/breast-cancer/experiment-populationgpu-{}-{}/'

## Start the experiment

In [ ]:
for pop_size in population_points:
    for iteration_no in range(5):
        my_random_seed += 1
        random.seed(my_random_seed)
        start_time = datetime.now()
        
        print("################################################")
        print("################################################")
        print("Starting population {} iteration {}".format(pop_size, iteration_no))
        print("Started at {}".format(start_time.strftime("%m/%d/%Y, %H:%M:%S")))
        print("################################################")
        print("################################################")
        
        
        config = deepcopy(base_config)
        config.pop_size = pop_size
        
        saveLocation = saveLocationTemplate.format(pop_size, iteration_no)
        
        p = instantiate_population(config, xs, ys, saveLocation)
        # Run for up to nGenerations generations.
        winner = p.run(eval_genomes, maxNGenerations)
        
        g = p.best_genome

        
        end_time = datetime.now()
        
        p.reporters.reporters[2].save_checkpoint(p.config, p.population, p.species, str(p.generation) + "-final")  
        
        winner_net = neat.nn.FeedForwardNetwork.create(winner, config)

        results = []
        for xi, xo in zip(xs, ys):
            output = winner_net.activate(xi)
            results.append([xi, xo, output])

        df = pd.DataFrame(results)
        df.to_csv(os.path.join(saveLocation, 'results.csv'))
        
        ancestry = p.reporters.reporters[3].trace_ancestry_of_species(g.key, p.reproduction.ancestors) 

        ancestors = {
            k: v['genome'] for k, v in p.reporters.reporters[3].ancestry.items()
        }
    
        
        ## Save all of these to disc
        filename = 'fullStatus.xplnt'
        print("Saving checkpoint to {0}".format(filename))

        with gzip.open(os.path.join(saveLocation, filename), 'w', compresslevel=5) as f:
            data = (p, g, ancestry, ancestors, random.getstate())
            pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
#         visualize.create_ancestry_video(p.config, 
#                                         g, 
#                                         ancestry, 
#                                         ancestors, 
#                                         p.reporters.reporters[1], 
#                                         pathname=saveLocation)
        print("################################################")
        print("################################################")
        print("Have finished population {} iteration {}".format(pop_size, iteration_no))
        print("Started at {}".format(start_time.strftime("%m/%d/%Y, %H:%M:%S")))
        print("The time is {}".format(end_time.strftime("%m/%d/%Y, %H:%M:%S")))
        print("################################################")
        print("################################################")
    

################################################
################################################
Starting population 2 iteration 0
Started at 07/11/2019, 03:49:07
################################################
################################################

 ****** Running generation 0 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(3.8294, grad_fn=<DivBackward0>)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([569])) that is different to the input size (torch.Size([569, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Population's average fitness: 0.24651 stdev: 0.01864
Best fitness: 0.26515 - size: (1, 30) - species 1 - id 1
ending generation %s
Average adjusted fitness: 0.019
Mean genetic distance 1.742, standard deviation 1.232
Population of 2 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0     2      0.3    0.019     0
Total extinctions: 0
Generation time: 194.398 sec

 ****** Running generation 1 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(3.8294, grad_fn=<DivBackward0>)
Population's average fitness: 0.24651 stdev: 0.01864
Best fitness: 0.26515 - size: (1, 30) - species 1 - id 1
ending generation %s
Average adjusted fitness: 0.019
Mean genetic distance 1.742, standard deviation 1.232
Population of 2 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1     2      0.3    0.019     1
Total extinctions: 0
Gen

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(3.8294, grad_fn=<DivBackward0>)
Population's average fitness: 0.24651 stdev: 0.01864
Best fitness: 0.26515 - size: (1, 30) - species 1 - id 1
ending generation %s
Average adjusted fitness: 0.019
Mean genetic distance 1.742, standard deviation 1.232
Population of 2 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   13     2      0.3    0.019    13
Total extinctions: 0
Generation time: 186.344 sec (189.817 average)
Saving checkpoint to ./../../data/experiments/breast-cancer/experiment-populationgpu-2-0/checkpoint-13

 ****** Running generation 14 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(3.8294, grad_fn=<DivBackward0>)
Population's average fitness: 0.24651 stdev: 0.01864
Best fitness: 0.26515 - size: (1, 30) - species 1 - id 1
ending generation %s
Average adjusted fitness: 0

gen is 19
previous generation is [1]
skey is 1
gen is 18
previous generation is [1]
skey is 1
gen is 17
previous generation is [1]
skey is 1
gen is 16
previous generation is [1]
skey is 1
gen is 15
previous generation is [1]
skey is 1
gen is 14
previous generation is [1]
skey is 1
gen is 13
previous generation is [1]
skey is 1
gen is 12
previous generation is [1]
skey is 1
gen is 11
previous generation is [1]
skey is 1
gen is 10
previous generation is [1]
skey is 1
gen is 9
previous generation is [1]
skey is 1
gen is 8
previous generation is [1]
skey is 1
gen is 7
previous generation is [1]
skey is 1
gen is 6
previous generation is [1]
skey is 1
gen is 5
previous generation is [1]
skey is 1
gen is 4
previous generation is [1]
skey is 1
gen is 3
previous generation is [1]
skey is 1
gen is 2
previous generation is [1]
skey is 1
gen is 1
previous generation is [1]
skey is 1
gen is 0
previous generation is [1]
skey is 1
have calculated the ancestry
Saving checkpoint to fullStatus.xplnt
###

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(5.2585, grad_fn=<DivBackward0>)
Population's average fitness: 0.17437 stdev: 0.01745
Best fitness: 0.19351 - size: (1, 29) - species 2 - id 4
ending generation %s
Average adjusted fitness: 0.017
Mean genetic distance 1.646, standard deviation 1.635
Population of 4 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    2     2      0.2    0.000     2
     2    2     2      0.2    0.035     1
Total extinctions: 0
Generation time: 368.001 sec (306.075 average)
Saving checkpoint to ./../../data/experiments/breast-cancer/experiment-populationgpu-2-1/checkpoint-2

 ****** Running generation 3 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(5.2585, grad_fn=<DivBackward0>)
Population's average fitness: 0.17437 stdev: 0.01745
Best fitness: 0.19351 - size: (1, 29) - species 2 - id 4
ending g